In [12]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000
)

word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [14]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000
)

char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [16]:
train_features = train_word_features
test_features = test_word_features

## Results for Word Vectors

In [11]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cross_val_scores = cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc')
    cv_score = np.mean(cross_val_scores)
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
submission.to_csv('submission.csv', index=False)

toxic [ 0.95945615  0.95558536  0.9563585 ]
CV score for class toxic is 0.9571333361206901
severe_toxic [ 0.98429415  0.98365371  0.98650718]
CV score for class severe_toxic is 0.9848183479396968
obscene [ 0.97990425  0.9808691   0.97893412]
CV score for class obscene is 0.9799024880153892
threat [ 0.98018912  0.97529937  0.97611075]
CV score for class threat is 0.9771997443932067
insult [ 0.97111208  0.96764007  0.96965114]
CV score for class insult is 0.9694677655668938
identity_hate [ 0.97024949  0.96352311  0.96811185]
CV score for class identity_hate is 0.9672948150871212
Total CV score is 0.9726360828538331


In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

## Results for Character Vectors

In [18]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cross_val_scores = cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc')
    cv_score = np.mean(cross_val_scores)
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.9692158784155342
CV score for class severe_toxic is 0.9875939067187743
CV score for class obscene is 0.9838696176374401
CV score for class threat is 0.9833767767089041
CV score for class insult is 0.9774264627693338
CV score for class identity_hate is 0.973942885694692
Total CV score is 0.9792375879907796
